In [92]:
%load_ext autoreload
%autoreload 2
import os
import pickle as pkl
import numpy as np
import numpy.random as npr
import numpy.linalg as npl
import pandas as pd
import sklearn
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import dvu  # for visualization
import imodels
import sys
sys.path.append('../..')

# from config.config import EASY_DATASETS, HARD_DATASETS
from sklearn.tree import DecisionTreeClassifier
from experiments.models.stablelinear import StableLinearClassifier
from experiments.models.stableskope import StableSkopeClassifier
from experiments.notebooks import viz
from experiments.util import get_comparison_result, get_clean_dataset, get_best_model_under_complexity, MODEL_COMPARISON_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# generate the data

In [83]:
np.random.seed(14)
n = 500
d = 30
num_rules = 5
X = np.random.randn(n, d)
rules = npr.choice(d, size=num_rules, replace=False)
threshes = np.zeros(d) # assumes all rules are greater than thresh
gt_weights = np.random.randn(num_rules).reshape(-1, 1)
gt_weights = gt_weights / npl.norm(gt_weights)
print(rules)

[15  0 28 12  5]


In [84]:
def pred_rules(X, rules, threshes, gt_weights, noise_sigma=0.01):
    
    # extract out rule binary features
    rule_features = np.zeros((X.shape[0], rules.size))
    for i in range(rules.size):
        rule_features[:, i] = X[:, rules[i]] > threshes[i]
    
    # define outcome
    y_reg = (rule_features @ gt_weights + noise_sigma * np.random.randn(X.shape[0], 1)).flatten()
    y_class = (y_reg > 0).astype(int)
        
    return y_reg, y_class

y_reg, y_class = pred_rules(X, rules, threshes, gt_weights)

# fit some classifiers

In [94]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_class, test_size=0.33, random_state=42)

In [95]:
models = [
    imodels.BoostedRulesClassifier(),
    imodels.FPLassoClassifier(),
    sklearn.tree.DecisionTreeClassifier()
]

In [96]:
for model in models:
    model.fit(X_train, y_train)

In [86]:
m.fit(X, y_class)

BoostedRulesClassifier()

In [58]:
m = StableLinearClassifier(weak_learners=[,
                                          imodels.BoostedRulesClassifier],
                           max_complexity=5)

In [59]:
m.fit(X, y_class)

/system/linux/anaconda3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


TypeError: fit() missing 1 required positional argument: 'y'